In [2]:
pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
from sklearn.decomposition import PCA

file_path = "/content/drive/MyDrive/Colab Notebooks/merged-dataset.txt"
with open(file_path, "r") as file:
    data = [line.strip() for line in file if line.strip()]

train_ratio = 0.2
train_size = int(len(data) * train_ratio)

# Split the data
train_data = data[train_size:]
validation_data = data[:train_size]

# Save the splits
with open("train_dataset.txt", "w", encoding="utf-8") as train_file:
    train_file.writelines(train_data)

with open("validation_dataset.txt", "w", encoding="utf-8") as validation_file:
    validation_file.writelines(validation_data)

In [5]:
import numpy as np
import os

# 1. Downloading GloVe embeddings
glove_file = "glove.6B.100d.txt"
if not os.path.exists(glove_file):
    !wget http://nlp.stanford.edu/data/glove.6B.zip  # Download the GloVe embeddings
    !unzip glove.6B.zip

--2025-01-05 05:49:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-05 05:49:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-05 05:49:35--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [6]:
import numpy as np

# Load GloVe embeddings
embedding_file = "/content/glove.6B.100d.txt"  # Change to your GloVe file
glove_embeddings = {}
with open(embedding_file, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        glove_embeddings[word] = vector


In [7]:
!pip install torch # Install the torch module

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Map GloVe embeddings to GPT-2 vocabulary
embedding_dim = model.transformer.wte.weight.shape[1]  # Get the correct embedding dimension from the model
embedding_matrix = np.zeros((len(tokenizer), embedding_dim))
for i, token in enumerate(tokenizer.get_vocab()):
    word = tokenizer.decode([i]).strip()
    if word in glove_embeddings:
        # Project GloVe embeddings to the correct dimension using linear projection
        glove_embedding = glove_embeddings[word]
        # Create a projection matrix with the correct output dimension (embedding_dim)
        projection_matrix = np.random.randn(len(glove_embedding), embedding_dim)
        projected_embedding = glove_embedding @ projection_matrix  # Project the GloVe embedding
        embedding_matrix[i] = projected_embedding
    else:
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

# Replace the model's embedding layer
model.resize_token_embeddings(len(tokenizer))  # Ensure compatibility with tokenizer
with torch.no_grad():
    model.transformer.wte.weight = torch.nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
from datasets import Dataset
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):

    # Tokenize the examples
    tokenized_examples = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_special_tokens_mask=True # Return a special tokens mask
    )
    tokenized_examples["labels"] = tokenized_examples["input_ids"].copy()
    return tokenized_examples



train_dataset = Dataset.from_dict({"text": train_data})
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
print(train_dataset)
print(tokenized_dataset[0])


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 9
})
{'text': 'Research Training National Institutes of Health NIH Skip to main content U.S. Department of Health Human Services Search the NIH Website Virtual Tour Staff Directory En Espa ol Site Menu Home Health Information Health Care Providers Facilities Health Info Lines HealthCare.gov Science Education Resources NIH Clinical Research Trials and You Talking to Your Doctor More Search Health Topics Quick Links MedlinePlus Health Info NIH News in Health Wellness Toolkits Grants Funding Grants Home Page Find Funding Due Dates How to Apply About Grants Policy Compliance Grants News Blog Contracts Loan Repayment More Quick Links RePORT eRA Commons NIH Common Fund News Events News Releases Digital Media Kits Media Resources Media Contacts Images and B roll Events Social Media More Quick Links NIH News in Health NIH Research Matters NIH Record Research Training Medical Research Initiatives Science Highlights Science Education Research in NI

In [9]:

from transformers import TrainingArguments, Trainer



# Define a function to compute the loss
def compute_loss(model, inputs, return_outputs=False):
    """Computes the language modeling loss."""
    labels = inputs.get("labels")
    # forward pass
    outputs = model(**inputs)
    # If `return_outputs=True` return the predictions and the labels
    if return_outputs:
        return (outputs.loss, outputs.logits)
    return outputs.loss  # Return the loss

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=20,
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,
    logging_steps=10,
    report_to="none",
)
# Create a validation dataset similar to the training dataset
validation_dataset = Dataset.from_dict({"text": validation_data})
validation_dataset = validation_dataset.map(tokenize_function, batched=True) # Tokenize validation dataset

# Include the labels in the training dataset
#tokenized_dataset = tokenized_dataset.map(lambda examples: {'labels': examples['input_ids'][0]}, batched=False)
#validation_dataset = validation_dataset.map(lambda examples: {'labels': examples['input_ids'][0]}, batched=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    # Remove compute_loss argument
)

# Define a custom training step function using the compute_loss function
def train_step(model, inputs):
    """Custom training step to use the compute_loss function."""
    loss = compute_loss(model, inputs)
    loss.backward()
    return {"loss": loss.item()}

# Set the custom training step function for the Trainer
trainer.train_step = train_step



trainer.train()




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<ipython-input-9-690cbe472265>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,10.277060
2,10.113800,8.322973
3,10.113800,7.009166
4,7.994300,6.248058
5,7.994300,5.661175
6,6.528500,5.225736
7,6.528500,4.822916
8,5.484300,4.426298
9,5.484300,4.026575
10,4.720800,3.543650


TrainOutput(global_step=100, training_loss=4.99537425994873, metrics={'train_runtime': 404.739, 'train_samples_per_second': 0.445, 'train_steps_per_second': 0.247, 'total_flos': 17093842698240.0, 'train_loss': 4.99537425994873, 'epoch': 20.0})

In [10]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2")


('/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2/added_tokens.json')

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model
model_path = "/content/drive/MyDrive/Colab Notebooks/GLOVEnew_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Generate text
input_text = "Prophet Ibrahim AS as only then"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
attention_mask = torch.ones_like(input_ids)
output = model.generate(input_ids, attention_mask=attention_mask,max_length=50, no_repeat_ngram_size=2,num_return_sequences=1)

print("Generated Text:")
generated_text=tokenizer.decode(output[0], skip_special_tokens=False)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
Prophet Ibrahim AS as only then poll poll chemical polllegleg poll ½ poll cultural pollottiotti poll stationary poll frequencies poll prostate poll voter voter chemical chemicalotti mov mov poll movotti ½ ½leg chemicalhhhhhh pollvectorvector polleligible poll garg


In [12]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
references = "".join(validation_data).split()
candidate = generated_text.split()
smoothing_function = SmoothingFunction().method1
bleu_score1 = sentence_bleu(references, generated_text,weights=(1, 0, 0, 0), smoothing_function=smoothing_function)
bleu_score2 = sentence_bleu(references, generated_text,weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)
bleu_score3 = sentence_bleu(references, generated_text,weights=(0.3, 0.3, 0.3, 0), smoothing_function=smoothing_function)
bleu_score4 = sentence_bleu(references, generated_text,weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function)
print(f"BLEU Score 1: {bleu_score1}")
print(f"BLEU Score 2: {bleu_score2}")
print(f"BLEU Score 3: {bleu_score3}")
print(f"BLEU Score 4: {bleu_score4}")

BLEU Score 1: 0.17424242424242425
BLEU Score 2: 0.22291008109020524
BLEU Score 3: 0.23292848154178783
BLEU Score 4: 0.1500160063760122


In [13]:
!pip install rouge-score
from rouge_score import rouge_scorer
references = "".join(validation_data)
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
scores = scorer.score(references , generated_text)
print("ROUGE Scores:", scores)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3a9e7626901f9c7c621a1f7b505ef2c922eda1d6f851d2a2f5bf55f8a8440ca9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
ROUGE Scores: {'rouge1': Score(precision=0.05714285714285714, recall=0.0015625, fmeasure=0.003041825095057034), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.05714285714285714, recall=0.0015625, fmeasure=0.003041825095057034)}
